In [ ]:
from CoolProp.CoolProp import PropsSI

In [ ]:
def heat_power_demand(thermal_cond, thickness, insulator_area, desired_temperature, unpowered_temperature):
    heat_transfer_coeff = thermal_cond / thickness # U = k/L
    delta_temperature = desired_temperature - unpowered_temperature
    heat_demand = heat_transfer_coeff * insulator_area * delta_temperature
    return heat_demand

thermal_cond_material = {
    "Polyurethane Foam":0.022,
    "Reinforced concrete":2.3,
    "Concrete":0.11
}

room_size = [3,2,2] # m x,y,z
outside_temp = 15 + 273.15 # C
desired_temp = -10 + 273.15 # C
wall_thickness = 15 # cm
floor_thickness = 100 # cm

walls_area = room_size[0]*room_size[2]*2 + room_size[1]*room_size[2]
floor_area = room_size[0]*room_size[1]*2
total_area = walls_area + floor_area # m^2

hpd = heat_power_demand(thermal_cond_material["Polyurethane Foam"], wall_thickness/100, total_area, desired_temp, outside_temp)
print(f"Heat power demand: {hpd} kW")

In [ ]:

cond_temp = outside_temp + 20
evap_temp = desired_temp - 10

In [ ]:
#PropsSI("H", "T", evap_temp, "Q", 0, "R290")
#PropsSI("H", "T", -25, "Q", 0, "R134a")
#PropsSI("H", "P", 100000, "Q", 1, "R134a")

In [ ]:
outside_temp = 15 + 273.15 # C
desired_temp = -10 + 273.15 # C

cond_temp = outside_temp + 20
evap_temp = desired_temp - 10

fluid = "R290"
t_2 = evap_temp # T_evap (K)
t_4 = cond_temp # T_cond (K)
eta_s = 0.8 # Efficiency assumed to be 80%
heat_evap = 2e5 # Nominal heat power (W)

print("t_2: ",t_2)
print("t_4: ",t_4)

p_2 = PropsSI("P", "T", t_2, "Q", 1, fluid)
p_4 = PropsSI("P", "T", t_4, "Q", 0, fluid)

h_2 = PropsSI("H", "T", t_2, "Q", 1, fluid)
s_2 = PropsSI("S", "T", t_2, "Q", 1, fluid)

p_3 = p_4 # pressure remains same after compressor / before expansion
h_3s = PropsSI("H", "S", s_2, "P", p_3, fluid)
h_3 = h_2 + (h_3s - h_2) / eta_s # enthalpy considering compressor efficiency

h_4 = PropsSI("H", "T", t_4, "Q", 0, fluid)
h_1 = h_4 # enthalpy remains constant in expansion. refer to ph graph
p_1 = p_2 # pressure remains constant in evaporation.

# no calculamos s_1, s_3, s_4, t_1, t_3

m = heat_evap / (h_2 - h_1) # mass flow rate kg/s. Are these enthalpies correct? or should we get h_4, h_3
power = m * (h_3 - h_2) # compressor power
cop = abs(heat_evap) / power
heat_cond = abs(heat_evap) - power

print(cop)
print(power)
print(heat_cond)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

outside_temp = 15 + 273.15 # C
desired_temp = -10 + 273.15 # C

cond_temp = outside_temp + 20
evap_temp = desired_temp - 10

evap_temp_list = np.arange(evap_temp - 10 , evap_temp + 10 , 2)
#cond_temp_list = np.arange(evap_temp - 10 , evap_temp + 10 , 2)
cop_list = []
#cond_temp = -20+273

for evap_temp in (evap_temp_list):
    fluid = "R290"
    t_2 = evap_temp # T_evap (K)
    t_4 = cond_temp # T_cond (K)
    eta_s = 0.8 # Efficiency assumed to be 80%
    heat_evap = 2e5 # Nominal heat power (W)

    p_2 = PropsSI("P", "T", t_2, "Q", 1, fluid)
    p_4 = PropsSI("P", "T", t_4, "Q", 0, fluid)

    h_2 = PropsSI("H", "T", t_2, "Q", 1, fluid)
    s_2 = PropsSI("S", "T", t_2, "Q", 1, fluid)

    p_3 = p_4 # pressure remains same after compressor / before expansion
    h_3s = PropsSI("H", "S", s_2, "P", p_3, fluid)
    h_3 = h_2 + (h_3s - h_2) / eta_s # enthalpy considering compressor efficiency

    h_4 = PropsSI("H", "T", t_4, "Q", 0, fluid)
    h_1 = h_4 # enthalpy remains constant in expansion. refer to ph graph
    p_1 = p_2 # pressure remains constant in evaporation.

    # no calculamos s_1, s_3, s_4, t_1, t_3

    m = heat_evap / abs(h_2 - h_1) # mass flow rate kg/s. Are these enthalpies correct? or should we get h_4, h_3
    power = m * abs(h_3 - h_2) # compressor power
    cop = abs(heat_evap) / power
    heat_cond = abs(heat_evap) - power

    #print(cop)
    #print(power)
    #print(heat_cond)
    cop_list.append(cop)

print("cond_temp: ",cond_temp)
print("evap_temp_list: ",evap_temp_list)
print("cop_list: ",cop_list)
plt.plot(evap_temp_list, cop_list)

# Plotting multiple lines on a single plot
#plt.plot(x, y1, label='Sin(x)', color='b')
#plt.plot(x, y2, label='Cos(x)', color='r', linestyle='--')
 
# Adding labels and title
plt.xlabel('Evaporated T [K]')
plt.ylabel('COP')
plt.title('COP vs Evaporated T')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

outside_temp = 15 + 273.15 # C
desired_temp = -10 + 273.15 # C

cond_temp = outside_temp + 20
evap_temp = desired_temp - 10

#evap_temp_list = np.arange(evap_temp - 10 , evap_temp + 10 , 2)
cond_temp_list = np.arange(cond_temp - 10 , cond_temp + 10 , 2)

cop_list = []
#cond_temp = -20+273
#print(*cond_temp_list)

for cond_temp in (cond_temp_list):
    fluid = "R290"
    t_2 = evap_temp # T_evap (K)
    t_4 = cond_temp # T_cond (K)
    eta_s = 0.8 # Efficiency assumed to be 80%
    heat_evap = 2e5 # Nominal heat power (W)

    p_2 = PropsSI("P", "T", t_2, "Q", 1, fluid)
    p_4 = PropsSI("P", "T", t_4, "Q", 0, fluid)

    h_2 = PropsSI("H", "T", t_2, "Q", 1, fluid)
    s_2 = PropsSI("S", "T", t_2, "Q", 1, fluid)

    p_3 = p_4 # pressure remains same after compressor / before expansion
    h_3s = PropsSI("H", "S", s_2, "P", p_3, fluid)
    h_3 = h_2 + (h_3s - h_2) / eta_s # enthalpy considering compressor efficiency

    h_4 = PropsSI("H", "T", t_4, "Q", 0, fluid)
    h_1 = h_4 # enthalpy remains constant in expansion. refer to ph graph
    p_1 = p_2 # pressure remains constant in evaporation.

    # no calculamos s_1, s_3, s_4, t_1, t_3

    m = heat_evap / abs(h_2 - h_1) # mass flow rate kg/s. Are these enthalpies correct? or should we get h_4, h_3
    power = m * abs(h_3 - h_2) # compressor power
    cop = abs(heat_evap) / power
    heat_cond = abs(heat_evap) - power

    #print(cop)
    #print(power)
    #print(heat_cond)
    cop_list.append(cop)


print("evap_temp: ",evap_temp)
print("cond_temp_list: ",cond_temp_list)
print("cop_list: ",cop_list)
plt.plot(cond_temp_list, cop_list)

# Plotting multiple lines on a single plot
#plt.plot(x, y1, label='Sin(x)', color='b')
#plt.plot(x, y2, label='Cos(x)', color='r', linestyle='--')
 
# Adding labels and title
plt.xlabel('Condensated T [K]')
plt.ylabel('COP')
plt.title('COP vs Condensated T')

Now, let's plot both the condensated T° and the evaporated T° in the same 3D graph to compare values:

In [ ]:
import pandas as pd

def run_forward(fluid, t_2, t_4, eta_s, heat):
    print("t_2: ", t_2)
    print("t_4: ", t_4)
    p_2 = PropsSI("P", "T", t_2, "Q", 1, fluid)
    p_4 = PropsSI("P", "T", t_4, "Q", 0, fluid)

    h_2 = PropsSI("H", "T", t_2, "Q", 1, fluid)
    s_2 = PropsSI("S", "T", t_2, "Q", 1, fluid)

    p_3 = p_4
    h_3s = PropsSI("H", "S", s_2, "P", p_3, fluid)
    h_3 = h_2 + (h_3s - h_2) / eta_s

    h_4 = PropsSI("H", "T", t_4, "Q", 0, fluid)

    m = heat / (h_4 - h_3)
    power = m * (h_3 - h_2)
    cop = abs(heat) / power
    heat_evap = abs(heat) - power

    print("m: ", m)
    print("cop: ", cop)
    
    return m, power, cop, heat_evap

fluid = "R290"
t_2 = 283.15
t_4 = 333.15
eta_s = 0.8
heat = -1e6

#m, power, cop, heat_evap = run_forward(fluid, t_2, t_4, eta_s, heat)
#round(cop, 2)


t_2_range = np.linspace(-15, 15, 31)
t_4_range = np.linspace(40, 70, 31)

cop_parametric = pd.DataFrame(index=t_2_range, columns=t_4_range)
#print("FIRST cop_parametric: ",cop_parametric)

for t_2 in t_2_range:
    _, _, cop, _ = run_forward(fluid, t_2 + 273.15, t_4_range + 273.15, eta_s, heat)
    cop_parametric.loc[t_2] = cop
    #print("  cop_parametric[",str(t_2),"]: ",cop_parametric)

#print("LAST cop_parametric: ",cop_parametric)
